In [1]:
import sys

sys.path.append('..')

from utils import (load_data, CLASSIFICATION, REGRESSION, coffe_norm, get_regression_model,
                  save_conf_matrix, error_angle, eval_model, get_generator_wrapper)
import numpy as np
import os
import matplotlib.pyplot as plt 
import sys
from glob import glob
import json
import tensorflow as tf

from makiflow import set_main_gpu

set_main_gpu(2)

In [2]:
EXP_FOLDER = 'exp_regression_kfold_4'
LR = 8e-3
BATCH_SIZE = 64
EXP_FOLDERS = glob(os.path.join(EXP_FOLDER, '*'))
EPOCHS = 30

THR_PRED_LIST = [0.01, 0.05, 0.1, 0.15, 0.2]

In [ ]:
for path_to_exp in EXP_FOLDERS:
    # At start of each experiments - create new dataset (shuffle it)
    Xtrain, Ytrain, Xtest, Ytest, pred2params, config_data = load_data(
        path_exp_folder=path_to_exp,
        path_to_data='..',
        use_saved=True,
        size_hw=(336, 336),
        test_div=0.8,
        data_type=REGRESSION,
        filename_config='config_data'
    )

    Ytrain = np.array(Ytrain).astype(np.float32).reshape(-1, 1)
    Ytest = np.array(Ytest).astype(np.float32).reshape(-1, 1)
    
    # Create model
    model, trainer, opt, global_step, sess = get_regression_model(
        size_hw=(336, 336), 
        batch_size=BATCH_SIZE, 
        size_dataset=len(Ytrain),
        lr=LR
    )
    
    gen = get_generator_wrapper(Xtrain, Ytrain, BATCH_SIZE)
    accur_list = []

    for epoch_num in range(EPOCHS):
        # Each epoch:
        # Train -> eval -> create folder -> save weights/arch -> save results
        print('ep: ', epoch_num)
        info1 = trainer.fit_generator(
            gen, optimizer=opt, epochs=1, 
            iter=len(Xtrain)//(2*BATCH_SIZE), print_period=150, global_step=global_step
        )
        predictions = eval_model(
            model, 
            Xtest, Ytest, 
            BATCH_SIZE, 
            calc_mean=False, is_classic=False, return_predict_only=True
        ).reshape(-1, 1)
        # {value_thr: pred_acc}
        json_pred_info = dict()
        for thr_value in THR_PRED_LIST:
            abs_pr_and_y = np.abs(predictions - Ytest[:len(predictions)])
            pred_acc = np.mean(abs_pr_and_y <= thr_value)
            json_pred_info[str(thr_value)] = pred_acc
        print('acc test: ', json_pred_info)
        accur_list.append(json_pred_info)
        # Create folder
        path_to_s_epoch = os.path.join(
            path_to_exp, 
            f'acc_{round(json_pred_info[str(THR_PRED_LIST[-1])], 2)}_ep_{epoch_num}_exp'
        )
        os.makedirs(path_to_s_epoch, exist_ok=True)
        # acc info - write into json
        with open(os.path.join(path_to_s_epoch, 'acc_data.json'), 'w') as fp:
            json.dump(json_pred_info, fp)
        # Save weights/arch
        model.save_weights(os.path.join(path_to_s_epoch, 'weights.ckpt'))
        model.save_architecture(os.path.join(path_to_s_epoch, 'model.json'))
        # Save results
        """
        save_conf_matrix(
            Ytest, predictions, 
            os.path.join(path_to_s_epoch, 'conf_matrix'),
            save_np=True
        )
        error_angle(
            os.path.join(path_to_s_epoch, 'error_angle_all_class'), 
            config_data, Ytest, predictions,
            save_np=True
        )
        """
    fig = plt.figure(figsize=(8, 8))
    for thr_value in THR_PRED_LIST:
        pred_taken = [single[str(thr_value)] for single in accur_list]
        plt.plot(pred_taken)
    plt.legend(list(map(lambda x: str(x), THR_PRED_LIST)))
    fig.savefig(os.path.join(path_to_exp, 'acc.png'))
    # Clear plt figures
    plt.close('all')
    # close session and reset graph
    sess.close()
    tf.reset_default_graph()

  0%|          | 65/34020 [00:00<00:52, 644.15it/s]

Read data from config file:  config_data


100%|██████████| 3780/3780 [00:05<00:00, 645.74it/s]

train :  34020
test:  3780







Loss is built.


ep:  0



...

  0%|          | 0/265 [00:00<?, ?it/s]

Weights are saved to exp_regression_kfold_4/1_exp/acc_0.97_ep_18_exp/weights.ckpt
Model's architecture is saved to exp_regression_kfold_4/1_exp/acc_0.97_ep_18_exp/model.json.
ep:  19


 57%|█████▋    | 150/265 [02:42<01:58,  1.03s/it]

Epoch: 0
Regression_Head_1: 0.0435570 
Training_Loss: 0.0435570 




100%|██████████| 265/265 [04:35<00:00,  1.04s/it]


In [ ]:
%matplotlib inline

In [ ]:
plt.imshow(Xtest[-100])

In [ ]:
Ytest[-100]

In [ ]:
len(acc_test)

In [ ]:
len(predictions)

In [ ]:
1

In [ ]:
2